### Co zrobione
  ### 'Data_pierwszej_rejestracji'
    - Naprawienie formatowania 'Data_pierwszej_rejestracji'
    - Rozdzielenie 'Data_pierwszej_rejestracji' na kolumny 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji'
    - Zmiana nieprawindłowych lat z 'Rok_pierwszej_rejestracji' poprzez iterowanie po indeksach roku i zamianie ich na wartości o odpowiadającym indeksie z 'Rok_produkcji'
    - Usunięcie atrybutu 'Data_pierwszej_rejestracji' (Warto się zastanowić czy zostawić usunięte, czy może zrobić merge 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji')
  ### 'Generacja_pojazdu'
    - Chyba nie trzeba czyścić, braki końcowego roku zdają się być schematyczne a nie wybiórcze.  Możliwe jednak że coś przegapiłem

  ### 'Cena' i 'Waluta'
    - Wartości ceny wyrażonej w EUR zostały przekonwertowane na PLN przy pomocy API NBP wedle średniego kursu walut dnia wystawienia pojazdu lub najbliższej dostępnej dacie wśród pozostałych wartości z EUR
    - Atrybut 'Waluta' usunięty skoro  wszystkie wartości wyrażone są w PLN

  ### 'Wyposazenie'
    - Wykonany One Hot Encoding wyposażenia
    - Usunięty atrybut 'Wyposazenie'

In [179]:
from enum import unique

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [180]:
df_car_sales_train_data = pd.read_csv('Data/sales_ads_train.csv',
    dtype={
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'

},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  int64         
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [181]:
df_car_sales_test_data = pd.read_csv('Data/sales_ads_test.csv',
    dtype={
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'

},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 72907 entries, 1 to 72907
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Waluta                      72907 non-null  category      
 1   Stan                        72907 non-null  category      
 2   Marka_pojazdu               72907 non-null  string        
 3   Model_pojazdu               72907 non-null  string        
 4   Wersja_pojazdu              48394 non-null  string        
 5   Generacja_pojazdu           51734 non-null  string        
 6   Rok_produkcji               72907 non-null  float64       
 7   Przebieg_km                 72577 non-null  float64       
 8   Moc_KM                      72678 non-null  float64       
 9   Pojemnosc_cm3               72221 non-null  float64       
 10  Rodzaj_paliwa               72907 non-null  category      
 11  Emisja_CO2                  32946 non-null  float64       


# UWAGA!!
Łączenie danych testowych i treningowych tylko na czas czyszczenia stringów i formatowań!!!

In [182]:
df_car_sales_test_data['Cena'] = pd.Series()
df_car_sales = pd.concat([df_car_sales_train_data, df_car_sales_test_data], ignore_index = True)
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208304 entries, 0 to 208303
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  object        
 1   Waluta                      204928 non-null  category      
 2   Stan                        204982 non-null  category      
 3   Marka_pojazdu               204953 non-null  string        
 4   Model_pojazdu               204995 non-null  string        
 5   Wersja_pojazdu              135730 non-null  string        
 6   Generacja_pojazdu           145471 non-null  string        
 7   Rok_produkcji               204897 non-null  float64       
 8   Przebieg_km                 203971 non-null  float64       
 9   Moc_KM                      204342 non-null  float64       
 10  Pojemnosc_cm3               202932 non-null  float64       
 11  Rodzaj_paliwa               204894 non-

In [183]:
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Skrzynia_biegow,Typ_nadwozia,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie
0,13900,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,...,Manual,minivan,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air..."
1,25900,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,...,Manual,station_wagon,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air..."
2,35900,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,...,Manual,minivan,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ..."
3,5999,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,...,Manual,compact,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air..."
4,44800,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,...,Manual,NaN,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208299,NaN,PLN,Used,Volkswagen,Tiguan,2.0 TSI BMT 4Mot Highline DSG,II (2016-),2016.0,53000.0,220.0,...,Automatic,SUV,5.0,black,Poland,Yes,20/12/2020,2021-04-29,"Sąsiedzka 4 - 03-168 Warszawa, Białołęka (Polska)","['ABS', 'Electric front windows', 'Drivers air..."
208300,NaN,PLN,Used,Ford,Kuga,2.0 TDCi Trend,I (2008-2012),2009.0,188132.0,136.0,...,Manual,SUV,5.0,black,Germany,NaN,NaN,2021-04-30,ul. Małkińska 184 (między CPN a stacją kontrol...,"['ABS', 'Rain sensor', 'Electric front windows..."
208301,NaN,PLN,Used,Toyota,Avensis,<NA>,III (2009-),2016.0,76786.0,143.0,...,Manual,station_wagon,5.0,blue,Poland,Yes,NaN,2021-04-29,ALEJA WOJSKA POLSKIEGO 68/76 - 42-200 Częstoch...,"['ABS', 'Lane assistant', 'Central locking', '..."
208302,NaN,PLN,Used,Renault,Trafic,2.0 dCi 115 Combi L1H1,II (2001-2014),2009.0,204300.0,114.0,...,Manual,minivan,4.0,blue,NaN,NaN,NaN,2021-05-01,Gaj ( 3km Od Krakowa Przy Zakopiance)Myślenick...,"['ABS', 'Electric front windows', 'Drivers air..."


### Czyszczenie i konwersja 'Data_pierwszej_rejestracji' do typu 'datetetime64'

In [184]:
df_car_sales['Data_pierwszej_rejestracji']

0                NaN
1         16/06/2010
2                NaN
3         27/11/2007
4         20/05/2013
             ...    
208299    20/12/2020
208300           NaN
208301           NaN
208302           NaN
208303    17/03/2021
Name: Data_pierwszej_rejestracji, Length: 208304, dtype: object

Zakładając brak pomyłek:

 - najstarsze auto zostało wyprodukowane w 1923, lecz dane testowe zawierają w sobie nawet takie z 1915
 - dane pochodzą z 2021

In [185]:
df_car_sales['Rok_produkcji'].describe()

count    204897.000000
mean       2012.098323
std           7.002089
min        1915.000000
25%        2008.000000
50%        2013.000000
75%        2017.000000
max        2021.000000
Name: Rok_produkcji, dtype: float64

'Data_pierwszej_rejestracji' zawiera w sobie zarówno błędne formatowanie jak i rok rejestracji

In [186]:
df_car_sales['Data_pierwszej_rejestracji'] = df_car_sales['Data_pierwszej_rejestracji'].str.replace('//', '/')

In [187]:
# Dzielenie daty rejestracji na kolumny dnia, miesiąca i roku
first_registration_expanded_data = df_car_sales['Data_pierwszej_rejestracji'].str.split('/', expand=True).rename(columns={0: 'Dzien_pierwszej_rejestracji', 1: 'Miesiac_pierwszej_rejestracji', 2: 'Rok_pierwszej_rejestracji'})

In [188]:
# Łączenie datych z kolumną roku produkci pojazdu dla dalszej analizy
df_car_sales_with_expanded_registration_dates = pd.concat([df_car_sales['Rok_produkcji'], first_registration_expanded_data], axis=1)
df_car_sales_with_expanded_registration_dates

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
0,2005.0,NaN,NaN,NaN
1,2010.0,16,06,2010
2,2015.0,NaN,NaN,NaN
3,2007.0,27,11,2007
4,2013.0,20,05,2013
...,...,...,...,...
208299,2016.0,20,12,2020
208300,2009.0,NaN,NaN,NaN
208301,2016.0,NaN,NaN,NaN
208302,2009.0,NaN,NaN,NaN


In [189]:
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < 1940
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

# Zastąpienie pierwszych 2 cyfr roku cyframi z roku produkcji zdaje się że rozwiąże większość konfliktów

,Rok_produkcji,Rok_pierwszej_rejestracji
1726,2006.0,0012
1889,2016.0,0017
2264,1998.0,0098
2743,1998.0,0098
3998,2000.0,0200
...,...,...
200136,2019.0,0019
202137,2018.0,0018
202825,2008.0,-0001
204365,2004.0,0204


In [190]:
# Dodać do listy zakupów
df_car_sales.iloc[32583]

Cena                                                1300000
Waluta                                                  PLN
Stan                                                   Used
Marka_pojazdu                                   Rolls-Royce
Model_pojazdu                                       Phantom
Wersja_pojazdu                                         <NA>
Generacja_pojazdu                                      <NA>
Rok_produkcji                                        1936.0
Przebieg_km                                         40355.0
Moc_KM                                                  NaN
Pojemnosc_cm3                                        3499.0
Rodzaj_paliwa                                      Gasoline
Emisja_CO2                                              NaN
Naped                                           Rear wheels
Skrzynia_biegow                                      Manual
Typ_nadwozia                                      city_cars
Liczba_drzwi                            

In [191]:
# Usuwanie ujemnych wartości z roku
df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'] = (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].
                                                                              str.replace('-', ''))

In [192]:
# Wyświetlanie danych gdzie rok pierwszej rejestracji jest wcześniejszy niż rok produkcji
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < df_car_sales_with_expanded_registration_dates['Rok_produkcji'].astype(float)
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

,Rok_produkcji,Rok_pierwszej_rejestracji
821,1998.0,1997
1088,2011.0,2010
1423,2010.0,2009
1726,2006.0,0012
1889,2016.0,0017
...,...,...
206853,2020.0,2019
206917,2013.0,0013
207056,2005.0,2004
207315,2015.0,1970


In [193]:
# Sprawdzanie zcy Rok pierwszej rejestracji zawsze zawiera cyfry
mask = df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[\\d]') == False].index
df_car_sales_with_expanded_registration_dates.loc[mask]['Rok_pierwszej_rejestracji']


Series([], Name: Rok_pierwszej_rejestracji, dtype: object)

In [194]:
df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[0-9]{4}') == False]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji


In [195]:
test_data = df_car_sales_with_expanded_registration_dates

In [196]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
821,1998.0,28,05,1997
1088,2011.0,10,10,2010
1423,2010.0,12,12,2009
1726,2006.0,20,06,0012
1889,2016.0,10,08,0017
...,...,...,...,...
206917,2013.0,06,06,0013
207056,2005.0,09,09,2004
207315,2015.0,01,01,1970
207430,2007.0,12,09,2005


In [197]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
821,1998.0,28,05,1997
1088,2011.0,10,10,2010
1423,2010.0,12,12,2009
1726,2006.0,20,06,0012
1889,2016.0,10,08,0017
...,...,...,...,...
206917,2013.0,06,06,0013
207056,2005.0,09,09,2004
207315,2015.0,01,01,1970
207430,2007.0,12,09,2005


In [198]:
test_data.iloc[821]

Rok_produkcji                    1998.0
Dzien_pierwszej_rejestracji          28
Miesiac_pierwszej_rejestracji        05
Rok_pierwszej_rejestracji          1997
Name: 821, dtype: object

In [199]:
# Iterowanie po indeksach Rok_pierwszej_rejestracji i zastępowanie niewłaśiwyh wartości
for i in range(4):
    # Warunek
    mask = (
        df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].notnull()
        & df_car_sales_with_expanded_registration_dates['Rok_produkcji'].notnull()
        & (
            (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'])
            | (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) > 2021)
        )
    )

    # Sprawdzenie, czy są wiersze spełniające warunek
    if not df_car_sales_with_expanded_registration_dates[mask].empty:
        # Pobranie indeksów pasujących wierszy
        matching_indices = df_car_sales_with_expanded_registration_dates[mask].index

        # Iteracja przez pasujące wiersze
        for idx in matching_indices:
            # Pobranie roku produkcji i konwersja na string
            production_year = str(int(df_car_sales_with_expanded_registration_dates.loc[idx, 'Rok_produkcji']))
            # Pobranie pierwszej cyfry roku produkcji
            production_year_digit = production_year[i] if i < len(production_year) else ''

            # Pobranie roku rejestracji
            registration_year = test_data.loc[idx, 'Rok_pierwszej_rejestracji']
            # Podział roku rejestracji na głowę i ogon
            registration_year_head = registration_year[:i]
            registration_year_tail = registration_year[i + 1:]

            # Utworzenie nowej wartości roku rejestracji
            new_registration_year = registration_year_head + production_year_digit + registration_year_tail

            # Aktualizacja wartości w DataFrame
            test_data.at[idx, 'Rok_pierwszej_rejestracji'] = new_registration_year

# Sprawdzanie czy dalej występują niepoprawne wartości
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji


## Merge oczyszczony rok i miesiac pierwszej rejestracji z danymi z df_car_sales

In [200]:
df_car_sales['Rok_pierwszej_rejestracji'] = df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji']
df_car_sales['Miesiac_pierwszej_rejestracji']= df_car_sales_with_expanded_registration_dates['Miesiac_pierwszej_rejestracji']
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
0,13900,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,...,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
1,25900,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,...,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010,06
2,35900,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,...,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN
3,5999,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,...,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007,11
4,44800,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,...,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208299,NaN,PLN,Used,Volkswagen,Tiguan,2.0 TSI BMT 4Mot Highline DSG,II (2016-),2016.0,53000.0,220.0,...,5.0,black,Poland,Yes,20/12/2020,2021-04-29,"Sąsiedzka 4 - 03-168 Warszawa, Białołęka (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2020,12
208300,NaN,PLN,Used,Ford,Kuga,2.0 TDCi Trend,I (2008-2012),2009.0,188132.0,136.0,...,5.0,black,Germany,NaN,NaN,2021-04-30,ul. Małkińska 184 (między CPN a stacją kontrol...,"['ABS', 'Rain sensor', 'Electric front windows...",NaN,NaN
208301,NaN,PLN,Used,Toyota,Avensis,<NA>,III (2009-),2016.0,76786.0,143.0,...,5.0,blue,Poland,Yes,NaN,2021-04-29,ALEJA WOJSKA POLSKIEGO 68/76 - 42-200 Częstoch...,"['ABS', 'Lane assistant', 'Central locking', '...",NaN,NaN
208302,NaN,PLN,Used,Renault,Trafic,2.0 dCi 115 Combi L1H1,II (2001-2014),2009.0,204300.0,114.0,...,4.0,blue,NaN,NaN,NaN,2021-05-01,Gaj ( 3km Od Krakowa Przy Zakopiance)Myślenick...,"['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN


# Czyszczenie 'Generacja_pojazdu'

In [201]:
car_gen = df_car_sales['Generacja_pojazdu']
car_gen

0                    <NA>
1         III (2008-2016)
2           C (2011-2019)
3         Mk2 (2004-2011)
4             III (2009-)
               ...       
208299         II (2016-)
208300      I (2008-2012)
208301        III (2009-)
208302     II (2001-2014)
208303               <NA>
Name: Generacja_pojazdu, Length: 208304, dtype: string

In [202]:
# Podział 'Generacja_pojazdu' na 3 części
generation = (car_gen.str.split("(", expand=True).
              rename(columns={0:"Nazwa_generacji", 1:"Okres_produkcji"}))
# Usunięcie ')' z 'Okres_produkcji'
generation['Okres_produkcji'] = (generation['Okres_produkcji'].
                                 str.strip(")"))
# Rozdzielenie 'Okres_produkcji' na 'Poczatek_produkcji' i 'Koniec_produkcji
split_generation_years = (generation['Okres_produkcji'].
                          str.split("-", expand=True).
                          rename(columns={0: 'Poczatek_produkcji', 1:'Koniec_produkcji'})
                          )
# Usunięcie 'Okres_produkcji'
generation.drop(columns=['Okres_produkcji'], inplace=True)


#Przypisanie nowych kolumn do generation
generation['Poczatek_produkcji'] = split_generation_years['Poczatek_produkcji']
generation['Koniec_produkcji'] = split_generation_years['Koniec_produkcji']

# Pozbycie się white spaces
generation['Nazwa_generacji'] = generation['Nazwa_generacji'].str.replace("\\s", "", regex=True)
generation['Poczatek_produkcji'] = generation['Poczatek_produkcji'].str.replace("\\s", "", regex=True)
generation['Koniec_produkcji'] = generation['Koniec_produkcji'].str.replace("\\s", "", regex=True)

generation



,Nazwa_generacji,Poczatek_produkcji,Koniec_produkcji
0,<NA>,<NA>,<NA>
1,III,2008,2016
2,C,2011,2019
3,Mk2,2004,2011
4,III,2009,
...,...,...,...
208299,II,2016,
208300,I,2008,2012
208301,III,2009,
208302,II,2001,2014


In [203]:
generation["Marka_pojazdu"] = df_car_sales['Marka_pojazdu']

generation.value_counts()

Nazwa_generacji  Poczatek_produkcji  Koniec_produkcji  Marka_pojazdu
A                2008                2017              Opel             1872
III              2013                                  Škoda            1808
E90/E91/E92/E93  2005                2012              BMW              1713
B8               2014                                  Volkswagen       1671
J                2009                2015              Opel             1621
                                                                        ... 
CA0              1992                1995              Mitsubishi          1
II               2000                2003              Toyota              1
III              1999                2006              Mitsubishi          1
C10              1984                1988              Mitsubishi          1
C2               1976                1982              Audi                1
Name: count, Length: 685, dtype: int64

# Konwersja EUR -> PLN

In [204]:
# Slekcja dadnych z 'Waluta' == 'EUR'
euro_to_pln = df_car_sales.loc[df_car_sales['Waluta'] == "EUR"]
euro_to_pln

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
735,202950,EUR,Used,Mercedes-Benz,AMG GT,<NA>,<NA>,2019.0,15.0,500.0,...,5.0,black,NaN,NaN,NaN,2021-05-01,"Słoneczna - 05555 Racibory, piaseczyński, Mazo...",[],NaN,NaN
2852,35500,EUR,Used,Rover,Other,<NA>,<NA>,1965.0,7000.0,90.0,...,4.0,beige,NaN,NaN,NaN,2021-05-03,"Maltańska 1 lok 1-070 - 61-131 Poznań, Nowe Mi...",[],NaN,NaN
2876,11000,EUR,Used,Toyota,Land Cruiser,<NA>,II (1989-1997),1992.0,51609.0,160.0,...,5.0,gray,NaN,NaN,NaN,2021-05-03,"Calle Assagador De Ricardo, 57 - 03503 Madrid,...",[],NaN,NaN
3275,80000,EUR,Used,Mercedes-Benz,Other,<NA>,<NA>,1957.0,1.0,104.0,...,2.0,silver,NaN,NaN,NaN,2021-04-26,"Oborniki, obornicki, Wielkopolskie",[],NaN,NaN
5105,4150,EUR,Used,Ford,Transit Custom,320 L2H2 PKW VA,<NA>,2017.0,183000.0,130.0,...,4.0,red,Czech Republic,Yes,NaN,2021-04-30,"Kudowa-Zdrój, kłodzki, Dolnośląskie","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205334,NaN,EUR,Used,Lexus,GX,<NA>,<NA>,2018.0,25000.0,300.0,...,5.0,silver,NaN,NaN,NaN,2021-04-28,"Białystok, Podlaskie, Dojlidy",[],NaN,NaN
205631,NaN,EUR,Used,Citroën,C4 Picasso,1.6 BlueHDi Shine S&S EAT6,II (2013-),2018.0,38000.0,120.0,...,5.0,white,Belgium,Yes,10/12/2018,2021-04-26,"Wyrska 122 A - 43-170 Łaziska Górne, Łaziska Ś...","['ABS', 'Electric front windows', 'Drivers air...",2018,12
206198,NaN,EUR,Used,Volkswagen,Passat,2.0 TDI Comfortline DSG,B7 (2010-2014),2014.0,173633.0,140.0,...,5.0,beige,NaN,NaN,NaN,2021-04-29,"ul.Kartuska33 - 83-304 Przodkowo, kartuski, Po...",['Automatic air conditioning'],NaN,NaN
207227,NaN,EUR,Used,Volkswagen,Multivan,<NA>,<NA>,2007.0,225085.0,128.0,...,4.0,black,Germany,NaN,20/12/2007,2021-04-28,"Jankowice, rybnicki, Śląskie","['ABS', 'Electrically adjustable mirrors', 'Pa...",2007,12


In [205]:
import requests
import datetime
def get_exchange_rate(date: datetime, currency="EUR"):
    """
    Pobiera kurs EUR/PLN dla podanej daty.
    :param date: Data w formacie 'YYYY-MM-DD'.
    :param currency: Kod waluty (domyślnie 'EUR').
    :return: Kurs wymiany lub None, jeśli dane nie są dostępne.
    """
    url = f'https://api.nbp.pl/api/exchangerates/rates/a/{currency.lower()}/{date}/?format=json'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Sprawdza, czy odpowiedź jest poprawna
        data = response.json()
        return data["rates"][0]["mid"]  # Kurs średni
    except requests.exceptions.HTTPError as e:
        # print(f"Błąd podczas pobierania danych: {e}")
        return np.nan


In [206]:
# Selekcja unikalnych dat z 'Waluta' == 'EUR'
unique_dates = euro_to_pln['Data_publikacji_oferty'].unique()
unique_dates_df = pd.DataFrame(unique_dates, columns=["Date"])
unique_dates_df

,Date
0,2021-05-01
1,2021-05-03
2,2021-04-26
3,2021-04-30
4,2021-05-04
5,2021-04-28
6,2021-04-27
7,2021-05-02
8,2021-04-29
9,2021-04-24


In [207]:
# Tworzenie słownika z wartościami wymiany
dict_values = {}
list_of_exchange_rates = []
for i, index in enumerate(unique_dates.tolist()):
    exchange_rate = get_exchange_rate(str(index).split(" ")[0], 'EUR')
    dict_values[index] = (float(exchange_rate))
    list_of_exchange_rates.append(exchange_rate)
unique_dates_df['Exchange_rate'] = pd.Series(list_of_exchange_rates).bfill().ffill()
unique_dates_df

,Date,Exchange_rate
0,2021-05-01,4.5565
1,2021-05-03,4.5565
2,2021-04-26,4.5565
3,2021-04-30,4.5654
4,2021-05-04,4.5539
5,2021-04-28,4.5811
6,2021-04-27,4.5613
7,2021-05-02,4.5782
8,2021-04-29,4.5782
9,2021-04-24,4.5562


In [208]:

for row in unique_dates_df.itertuples():
    mask = euro_to_pln.loc[euro_to_pln['Data_publikacji_oferty'] == row[1]].index
    mask_len = len(mask)
    exchange_rate_numpy = pd.Series(np.full(mask_len, row[2]))
    df_car_sales.loc[mask, 'Cena'] = df_car_sales.loc[mask, 'Cena'] * row[2]
df_car_sales.drop(columns=['Waluta'], inplace=True)
df_car_sales

,Cena,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
0,13900,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,1998.0,...,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
1,25900,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,...,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010,06
2,35900,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,...,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN
3,5999,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,...,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007,11
4,44800,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,...,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208299,NaN,Used,Volkswagen,Tiguan,2.0 TSI BMT 4Mot Highline DSG,II (2016-),2016.0,53000.0,220.0,1984.0,...,5.0,black,Poland,Yes,20/12/2020,2021-04-29,"Sąsiedzka 4 - 03-168 Warszawa, Białołęka (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2020,12
208300,NaN,Used,Ford,Kuga,2.0 TDCi Trend,I (2008-2012),2009.0,188132.0,136.0,1997.0,...,5.0,black,Germany,NaN,NaN,2021-04-30,ul. Małkińska 184 (między CPN a stacją kontrol...,"['ABS', 'Rain sensor', 'Electric front windows...",NaN,NaN
208301,NaN,Used,Toyota,Avensis,<NA>,III (2009-),2016.0,76786.0,143.0,1995.0,...,5.0,blue,Poland,Yes,NaN,2021-04-29,ALEJA WOJSKA POLSKIEGO 68/76 - 42-200 Częstoch...,"['ABS', 'Lane assistant', 'Central locking', '...",NaN,NaN
208302,NaN,Used,Renault,Trafic,2.0 dCi 115 Combi L1H1,II (2001-2014),2009.0,204300.0,114.0,1995.0,...,4.0,blue,NaN,NaN,NaN,2021-05-01,Gaj ( 3km Od Krakowa Przy Zakopiance)Myślenick...,"['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN


# 'Wyposazenie'

In [209]:
# Wylistowanie wszystkich elementów z atrybutu 'Wyposazenie'
equipment_list = df_car_sales['Wyposazenie'].dropna().str.strip("[]").str.replace("'", "").str.split(", ")
# Zredukowanie listy do set'a z unikalnymi wartościami
unique_equipment = set([item for sublist in equipment_list for item in sublist if item])

# Stworzenie One Hot encoded atrybutu dla każdego elementu
for item in unique_equipment:
    df_car_sales[item] = df_car_sales['Wyposazenie'].str.contains(item, regex=False)
df_car_sales.drop(columns=['Wyposazenie'], inplace=True)
df_car_sales

,Cena,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Electrically adjustable seats,Panoramic roof,Speed limiter,Aftermarket radio,MP3,Air curtains,Front parking sensors,Heated front seats,Fog lights,LED lights
0,13900,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,1998.0,...,False,False,False,False,False,False,False,False,False,False
1,25900,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,...,False,False,False,False,False,False,False,True,True,False
2,35900,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,...,False,False,False,False,True,False,False,True,False,False
3,5999,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,...,False,False,False,False,False,False,False,False,False,False
4,44800,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208299,NaN,Used,Volkswagen,Tiguan,2.0 TSI BMT 4Mot Highline DSG,II (2016-),2016.0,53000.0,220.0,1984.0,...,False,False,True,False,True,True,True,True,True,True
208300,NaN,Used,Ford,Kuga,2.0 TDCi Trend,I (2008-2012),2009.0,188132.0,136.0,1997.0,...,False,False,False,False,True,True,False,False,True,False
208301,NaN,Used,Toyota,Avensis,<NA>,III (2009-),2016.0,76786.0,143.0,1995.0,...,False,False,False,False,True,True,False,False,True,True
208302,NaN,Used,Renault,Trafic,2.0 dCi 115 Combi L1H1,II (2001-2014),2009.0,204300.0,114.0,1995.0,...,False,False,False,False,False,False,False,False,False,False


In [210]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208304 entries, 0 to 208303
Data columns (total 94 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Cena                             135397 non-null  object        
 1   Stan                             204982 non-null  category      
 2   Marka_pojazdu                    204953 non-null  string        
 3   Model_pojazdu                    204995 non-null  string        
 4   Wersja_pojazdu                   135730 non-null  string        
 5   Generacja_pojazdu                145471 non-null  string        
 6   Rok_produkcji                    204897 non-null  float64       
 7   Przebieg_km                      203971 non-null  float64       
 8   Moc_KM                           204342 non-null  float64       
 9   Pojemnosc_cm3                    202932 non-null  float64       
 10  Rodzaj_paliwa                    204894 non-

# Lokalizacja_oferty

Note:
- zastnanowić się czy wartości numerycznych nie zamienić na przedziały
- Uwzględnić nieproporcjonalność danych, rozważyć zastosowanie Oversamplingu


- Kiedy NIE usuwać wierszy?

    Gdy braki są systematyczne (np. wszystkie samochody marki "Opel" mają brakującą Emisja_CO2) – usuwanie wprowadzi bias.

    Gdy dane są rzadkie (np. tylko 100 wierszy), a usunięcie wierszy znacząco zmniejszy próbkę.

    Gdy można uzupełnić braki (np. średni Przebieg_km dla danej marki/roku).
